In [25]:
import sys
print (sys.version)
spark.version

3.9.2 (default, Feb 28 2021, 17:03:44) 
[GCC 10.2.1 20210110]


'3.3.2'

# https://delta-io.github.io/delta-rs/
https://docs.rs/object_store/latest/object_store/azure/enum.AzureConfigKey.html#variants
https://github.com/Azure/Azurite?tab=readme-ov-file#supported-command-line-options
https://docs.delta.io/latest/index.html
https://docs.rs/deltalake/latest/deltalake/
https://delta-io.github.io/delta-rs/usage/installation/


In [29]:
import pandas as pd
import numpy as np
from deltalake import write_deltalake, DeltaTable

df = pd.DataFrame({"foo": np.random.randint(1, 10, 5), "bar": np.random.randint(1, 10, 5)})


# define container name
container = "data"

# define credentials
storage_options = {
    # "ACCOUNT_NAME": "devstoreaccount1",
    # "ACCESS_KEY": "Eby8vdM02xNOcqFlqUwJPLlmEtlCDXJ1OUzFT50uSRZ6IFsuFq2UVErCz4I6tq/K1SZFPTOtr/KBHBeksoGMGw==",
    "azure_storage_use_emulator" : "true",
}

# storage_options = {
#     "azure_storage_account_name": "devstoreaccount1",
#     "azure_storage_account_key": "Eby8vdM02xNOcqFlqUwJPLlmEtlCDXJ1OUzFT50uSRZ6IFsuFq2UVErCz4I6tq/K1SZFPTOtr/KBHBeksoGMGw==",
#     "azure_container_name" : "data",
#     # "azure_storage_use_emulator" : "true",
#     # "azure_skip_signature" : "true",
#     # "azure_storage_endpoint" : "http://azureite:10000/devstoreaccount1",
# }




write_deltalake(
    f"abfs://{container}/delta_table_pandas",
    df,
    # partition_by=['foo'],
    storage_options=storage_options,
    # schema_mode="overwrite",
    # overwrite_schema=True,
    mode="overwrite",
    overwrite_schema=True,
)

In [27]:
import json
from IPython.display import display, JSON
from deltalake import DeltaTable


dt = DeltaTable(f"abfs://{container}/delta_table_pandas",
          storage_options=storage_options)

pd = dt.to_pandas()

schema_json = dt.schema()

print("-----version--------")
print(dt.version())
print("----metadata-----")
print(dt.metadata())
print("---schema---")
# print(json.dumps(dt.schema(), indent=4))
# display(pd(schema_json["fields"]))
print(schema_json)
print("-----history-----")
history_list = dt.history()  # Get the history list
display(JSON(history_list)) 
pretty_json = json.dumps(history_list, indent=4)  # Convert to pretty JSON
print(pretty_json)  # Print nicely formatted JSON
# print(json.dumps(dt.history()))


pd

-----version--------
6
----metadata-----
Metadata(id: 26b16d13-2cb8-4c47-9544-6cd309f03cb1, name: None, description: None, partition_columns: [], created_time: 1738356229969, configuration: {})
---schema---
Schema([Field(foo, PrimitiveType("long"), nullable=True), Field(bar, PrimitiveType("long"), nullable=True)])
-----history-----


<IPython.core.display.JSON object>

[
    {
        "timestamp": 1738356229980,
        "operation": "CREATE TABLE",
        "operationParameters": {
            "mode": "ErrorIfExists",
            "protocol": "{\"minReaderVersion\":1,\"minWriterVersion\":1}",
            "metadata": "{\"configuration\":{},\"created_time\":1738356229969,\"description\":null,\"format\":{\"options\":{},\"provider\":\"parquet\"},\"id\":\"26b16d13-2cb8-4c47-9544-6cd309f03cb1\",\"name\":null,\"partition_columns\":[],\"schema\":{\"fields\":[{\"metadata\":{},\"name\":\"foo\",\"nullable\":true,\"type\":\"long\"}],\"type\":\"struct\"}}",
            "location": "abfs://data/delta_table_pandas"
        },
        "clientVersion": "delta-rs.0.10.0"
    },
    {
        "timestamp": 1738356363664,
        "operation": "WRITE",
        "operationParameters": {
            "partitionBy": "[]",
            "mode": "Overwrite"
        },
        "clientVersion": "delta-rs.0.10.0"
    },
    {
        "timestamp": 1738356686275,
        "operation": "WR

,foo,bar
0,4,8
1,6,2
2,6,2
3,1,5
4,9,6


In [ ]:
dt.delete (predicate = None)    #(storage_options=storage_options)
dt.vacuum()

In [ ]:
df = dt.to_pandas()
df_filtered = df[df["foo"] != 3]
from deltalake.writer import write_deltalake
write_deltalake(
    f"abfs://{container}/delta_table_pandas",
    df_filtered,
    mode="overwrite",
    storage_options=storage_options)
dt.vacuum()



